In [5]:
import numpy as np
import scipy.signal as signal

def autocorrelation_method(signal, order):
    """
    Estimate LPC coefficients using autocorrelation method.
    
    Args:
    - signal: input speech signal (1D numpy array)
    - order: LPC order (integer)
    
    Returns:
    - lpc_coeffs: LPC coefficients (1D numpy array)
    """
    # Compute autocorrelation coefficients
    R = np.correlate(signal, signal, mode='full')
    
    # Keep only positive lags
    R = R[len(R)//2:]
    
    # Create toeplitz matrix
    R_toeplitz = np.array([R[i:i+order] for i in range(order)])
    
    # Compute autocorrelation vector
    r = R[1:order+1]
    
    # Solve the normal equations to obtain LPC coefficients
    lpc_coeffs = np.linalg.solve(R_toeplitz, -r)
    
    return np.concatenate([[1], lpc_coeffs])

def lpcc(signal, order_lpcc, order_lpc):
    """
    Calculate LPCC coefficients from LPC coefficients.
    
    Args:
    - signal: input speech signal (1D numpy array)
    - order_lpcc: LPCC order (integer)
    - order_lpc: LPC order (integer)
    
    Returns:
    - lpcc_coeffs: LPCC coefficients (1D numpy array)
    """
    # Calculate LPC coefficients
    lpc_coeffs = autocorrelation_method(signal, order_lpc)
    
    # Ensure LPCC order does not exceed LPC order
    order_lpcc = min(order_lpcc, order_lpc)
    
    # Perform cepstral analysis to obtain LPCC coefficients
    lpcc_coeffs = lpc_coeffs.copy()
    for m in range(order_lpcc):  # Corrected loop range
        sum_term = np.sum([k * lpcc_coeffs[k] * lpcc_coeffs[k-m] for k in range(m+1)])  # Adjusted index
        lpcc_coeffs[m] += sum_term / (m + 1)  # Adjusted index
    return lpcc_coeffs

# Example usage:
# Assuming you have a speech signal stored in the variable 'signal' and you want to compute LPCC coefficients
order_lpcc = 12  # LPCC order
order_lpc = 10   # LPC order

# Generate a synthetic speech signal (example)
fs = 8000  # Sampling frequency
duration = 1  # Duration of the signal (seconds)
t = np.linspace(0, duration, fs * duration, endpoint=False)
f0 = 100  # Fundamental frequency (Hz)
signal = np.sin(2 * np.pi * f0 * t)

lpcc_coeffs = lpcc(signal, order_lpcc, order_lpc)
print("LPCC coefficients:", lpcc_coeffs)


LPCC coefficients: [ 1.00000000e+00 -2.33865614e-03 -1.66354695e+00  3.60261124e-03
  4.39444691e-04 -7.16203489e-03 -3.37435217e-03  1.28293769e-02
 -1.56611302e-02  1.87296881e-02 -3.71049494e-03]


In [7]:
import numpy as np
from scipy.io import wavfile
from python_speech_features import lpcc

from sklearn.mixture import GaussianMixture

# Function to extract LPCC features from audio file
def extract_lpcc_features(audio_file, win_len=0.025, win_step=0.01, num_filters=26, num_ceps=13):
    rate, signal = wavfile.read(audio_file)
    lpcc_features = lpcc(signal, rate, winlen=win_len, winstep=win_step, numcep=num_ceps, nfilt=num_filters)
    return lpcc_features

# Function to train GMM models for speaker identification
def train_gmm_models(lpcc_features, speaker_labels):
    gmm_models = {}
    unique_speakers = np.unique(speaker_labels)
    for speaker in unique_speakers:
        speaker_lpcc = lpcc_features[speaker_labels == speaker]
        gmm = GaussianMixture(n_components=5, covariance_type='diag')
        gmm.fit(speaker_lpcc)
        gmm_models[speaker] = gmm
    return gmm_models

# Function to predict speaker identity using GMM models
def predict_speaker(lpcc_features, gmm_models):
    speaker_predictions = []
    for features in lpcc_features:
        scores = {speaker: model.score([features]) for speaker, model in gmm_models.items()}
        predicted_speaker = max(scores, key=scores.get)
        speaker_predictions.append(predicted_speaker)
    return speaker_predictions

# Example usage:
# Assuming you have audio files stored in 'audio_files' and corresponding speaker labels in 'speaker_labels'

# Extract LPCC features from audio files
lpcc_features = np.concatenate([extract_lpcc_features(audio_file) for audio_file in audio_files])

# Train GMM models
gmm_models = train_gmm_models(lpcc_features, speaker_labels)

# Predict speaker identity
predicted_speakers = predict_speaker(lpcc_features, gmm_models)

# Print predicted speakers
print("Predicted Speakers:", predicted_speakers)


ModuleNotFoundError: No module named 'python_speech_features'